# Direkt Marketing mit Amazon SageMaker XGBoost

#### Supervised Learning: Binäre Klassifikation

Letztes Update: April, 2021

In [ ]:
!pip -q install --upgrade pip
!pip -q install sagemaker awscli boto3 smdebug --upgrade

___
# TEIL 1 - Herunterladen und Verarbeiten des Datensatzes

Zunächst müssen wir den [direkt marketing datensatz](https://archive.ics.uci.edu/ml/datasets/bank+marketing) von UCI's ML Repository downloaden. Anschließend werden wir die csv-Datei lesen, und für unser ML Modell vorbereiten. Dafür sollten wir erstmal verstehen, was für Daten wir zur Verfügung haben und was wir vorhersagen möchten (= Ziel Variable Y).

Der Datensatz enthält Informationen über Direkt Marketing Kampagnen einer portugiesischen Bank. Diese Kampagnen basieren auf Telefonaten in denen der Kunde ein Produkt (bank term deposit) kaufen kann. Der Ausgang des Telefonates ist in der Variable Y gespeichert und wird Ziel unserer Klassifikation (yes/no) sein. 

Wir möchten also mit unserem ML Model vorhersagen, ob ein Kunde positiv oder negativ auf das Marketing Angebot per Telefon reagiert. 

In [ ]:
import zipfile                                    
!wget -N https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
!unzip -o bank-additional.zip

In [ ]:
import numpy as np  
import pandas as pd 

In [ ]:
data = pd.read_csv('./bank-additional/bank-additional-full.csv', sep=';')
pd.set_option('display.max_columns', 100)     
data.head()

Auf den ersten Blick können wir sehen wir haben über 40k Zeilen mit verschiedenen Kundendaten und je 20 beschreibenden Variablen

In [ ]:
data.shape # (Anzahl Zeilen, Anzahl Spalten)

Zunächst sollten wir ein Gefühl für die Ziel-Variable y und deren Verteilung erhalten. Wir stellen fest, dass wir deutlich mehr "no" Beispiele haben und dementsprechend ein unausgewogenen Datensatz haben:

In [ ]:
data['y'].value_counts()

In [ ]:
print("Verhältnis von negativen zu positiven Werten: {}".format( data['y'].value_counts()[0]/data['y'].value_counts()[1]))

### Datensatz vorbereiten für Machine Learning Modelle
Den Datensatz zu bereinigen und zu transformieren ist ein signifikanter Teil eines jeden Machine Learning Projekts. Das Behandeln von Ausreißern, das Füllen von fehlenden Werten oder die Erstellung bzw. Transformation neuer Variablen sind nur Beispiele der möglichen Vorbereitungen. Wir tun dies mit dem Ziel wichtige Informationen herauszuarbeiten, die es dem Model einfacher machen zu lernen.

Zunächst können wir uns den Datensatz mit `.describe()` genauer ansehen und nach ersten Auffälligkeiten untersuchen:
Die Variable **pdays** (Anzahl der Tage seitdem letzten Kontakt zum Kunden) z.B. hat als Maximum 999 Tage - das könnte ein Platzhalter sein für den Fall, dass ein Kunde noch gar nicht kontaktiert worden ist. Daher macht es Sinn, eine neue Spalte **no_previous_contact** zu erstellen. Sämtliche Reihen mit `pdays==999` werden eine "0" erhalten, ansonsten "1". Anschließend können wir die Spalte **pdays** löschen.

In [ ]:
data.describe()

In [ ]:
data['no_previous_contact'] = np.where(data['pdays'] == 999, 1, 0)
data.drop(['pdays'], axis=1, inplace=True)

Die Spalte **job** hat viele Kategorien, die wir in aussagekräftigere Kategorien verwandeln können: 
Da es wahrscheinlich ist, dass die Berufstätigkeit des Kunden einen Einfluss auf die Zielvariable hat, können wir eine neue Variable **not_working** erstellen und dort "student", "retire" und "unemployed" zusammenfassen. 

In [ ]:
data['job'].value_counts()

In [ ]:
data['not_working'] = np.where(np.in1d(data['job'], ['student', 'retired', 'unemployed']), 1, 0)

Zu guter letzt kodieren wir die kategorialen Variablen in eine Reihe von Dummy Variablen mit pandas `get_dummies()`Funktion. Das bedeutet, dass wir aus jeder kategorialen Ausprägung einer Variable eine neue Spalte erstellen (auch bekannt als "One-Hot-Encoding".

In [ ]:
model_data = pd.get_dummies(data)  # Convert categorical variables to sets of indicators
model_data.head()

Nun haben wir aus jeder kategorialen Ausprägung einer Variable (wie z.B.**job**,**marital** etc.) viele neue Variablen erstellt und somit aus 21 schlussendlich 67 Variablen gemacht.

In [ ]:
model_data.shape

### Unterteilen des Datensatzes

Als nächstes splitten wir den Datensatz in drei seperate Datensätze: (70%), validation (20%) und test (10%). Während des Trainings werden wir versuchen die Performance beim Validation datensatz zu maximieren - sobald das Model fertig bereitgestellt ist, werden wir erneut die Performance auf dem Test datensatz evaluieren. 

Im Folgenden werden wir Amazon SageMaker's XGBoost algorithmus verwenden, welcher Daten entweder im libSVM oder CSV Format erwartet. Im Rahmen dieser Demo, werden wir uns auf das CSV-Format konzentrieren. Die erste Spalte der Datei mus sunsere Zielvariable y sein - ebenso sollte die datei keine Überschriften haben in den Spalten. 

In [ ]:
# Set the seed to 123 for reproductibility
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=123), 
                                                  [int(0.7 * len(model_data)), int(0.9*len(model_data))])  

# Drop the two columns for 'yes' and 'no' and add 'yes' back as first column of the dataframe
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
pd.concat([validation_data['y_yes'], validation_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('validation.csv', index=False, header=False)

# Dropping the target value, as we will use this CSV file for batch transform
test_data.drop(['y_no','y_yes'], axis=1).to_csv('test.csv', index=False, header=False)

Als nächstes müssen wir die Dateien in Amazon S3 Bucket hochladen:

In [ ]:
import sagemaker
import boto3, os

bucket = sagemaker.Session().default_bucket()                     
prefix = 'sagemaker/DEMO-xgboost-dm'

boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')

Zusätzlich müssen wir SageMaker sagen, wo die Trainings und Validierungs Daten gespeichert sind:

In [ ]:
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')
s3_data = {'train': s3_input_train, 'validation': s3_input_validation}

---
# Teil 2 - Model erstellen und trainieren

Wie anfangs erläutert, handelt es sich bei unserem use case um ein Klassifizierungsproblem. *XGBoost* ist ein beliebtes open-source Projekt für Gradient Boosting Trees und wurde erfolgreich verwendet in vielen machine learning Wettbewerben! SageMaker bietet eine entsprechendes managed framework an, welches wir im Folgenden benutzen werden:

In [ ]:
from sagemaker.estimator import Estimator
from sagemaker.debugger import rule_configs, Rule, DebuggerHookConfig, CollectionConfig


sess = sagemaker.Session()
region = boto3.Session().region_name    

container = sagemaker.image_uris.retrieve('xgboost', region,version='0.90-2')


xgb = Estimator(
    container,                                               # container des algorithmus XGBoost)
    role=sagemaker.get_execution_role(),                     # IAM Berechtigungen für SageMaker
    sagemaker_session=sess,                                  # Technisches Object
                                    
    input_mode='File',                                       # Kopieren des Datensatzes und anschließendes Trainieren 
    output_path='s3://{}/{}/output'.format(bucket, prefix),  # S3 Pfad, wo das Model gespeichert wird
                                    
    instance_count=1,                                  # Instance Spezifikationen
    instance_type='ml.m4.2xlarge',
                                    
    use_spot_instances=True,                           # Benutzung von spot instance
    max_run=300,                                       # Maximale Trainingszeit
    max_wait=600,                                      # Maximale Trainingszeit + Wartezeit auf spot instances.
                                    
    debugger_hook_config=DebuggerHookConfig(                 # Save training tensors
        s3_output_path='s3://{}/{}/debug'.format(bucket, prefix), 
        collection_configs=[
            CollectionConfig(
                name="metrics",
                parameters={
                    "save_interval": '1'
                }
            ),
            CollectionConfig(
                name="feature_importance",
                parameters={
                    "save_interval": '1'
                }
            )
        ],
    ),
    
    rules=[
        Rule.sagemaker(                                      # Konfiguration der Debugger Regel
            rule_configs.class_imbalance(),                  
            rule_parameters={
                "collection_names": "metrics"
            },
        ),
    ]
)

### Spezifizierung der hyperparameter
Jeder SageMaker integrierte Algorithmus hat verschiedene Hyperparameter, die je nach use case und Dateneigenschaften gesetzt werden müssen. Für XGBoost, gibt es folgende [Hyperparameter](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html).

Im Rahmen der Demo werden wir lediglich die folgenden drei Hyperparameter benutzen:
* Binäre Klassifikation: 'binary:logistic'.
* Zur Evaluierung der Performance verwenden wir die 'Area Under Curve' Metrik. 
* Wir möchten maximal 100 Runden trainieren- sollte die AUC Metrik sich in 10 Runden nicht verbessern, beenden wir das Training vorzeitig.  

In [ ]:
xgb.set_hyperparameters(
    objective='binary:logistic', 
    eval_metric='auc', 
    num_round=100,
    early_stopping_rounds=10
)

Nun sind alle Parameter spezifiziert und wir können das Training beginnen mit der `.fit()` Funktion und den entsprechenden S3 Pfäden zu den Training/Validation CSV Files.

In [ ]:
xgb.fit(s3_data)

Now, let's load the tensors saved during training, and plot them.

In [ ]:
import smdebug
from smdebug.trials import create_trial

s3_output_path = xgb.latest_job_debugger_artifacts_path()
trial = create_trial(s3_output_path)

Let's plot our metric over time.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import re

def get_data(trial, tname):
    """
    For the given tensor name, walks though all the iterations
    for which you have data and fetches the values.
    Returns the set of steps and the values.
    """
    tensor = trial.tensor(tname)
    steps = tensor.steps()
    vals = [tensor.value(s) for s in steps]
    return steps, vals

def plot_collection(trial, collection_name, regex='.*',figsize=(8, 6)):
    """
    Takes a `trial` and a collection name, and 
    plots all tensors that match the given regex.
    """
    fig, ax = plt.subplots(figsize=figsize)
    sns.despine()

    tensors = trial.collection(collection_name).tensor_names

    for tensor_name in sorted(tensors):
        if re.match(regex, tensor_name):
            steps, data = get_data(trial, tensor_name)
            ax.plot(steps, data, label=tensor_name)

    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax.set_title(collection_name)
    ax.set_xlabel('Iteration')

In [ ]:
plot_collection(trial, "metrics")

Als nächstes können wir die Feature Importance visualisieren:

In [ ]:
def plot_feature_importance(trial, importance_type="weight"):
    SUPPORTED_IMPORTANCE_TYPES = ["weight", "gain", "cover", "total_gain", "total_cover"]
    if importance_type not in SUPPORTED_IMPORTANCE_TYPES:
        raise ValueError(f"{importance_type} is not one of the supported importance types.")
    plot_collection(
        trial,
        "feature_importance",
        regex=f"feature_importance/{importance_type}/.*")

In [ ]:
plot_feature_importance(trial)

Variable 1 (job) und 5 (housing) sollten die wichtigsten Variablen sein.

___
# Teil 3 - Bereitstellen und Einsetzen des Models

## Variante 1: Vorhersagen mit einem SageMaker Endpunkt

Nun können wir unser trainiertes Model mit einem HTTPS Endpunkt! Anschließend können wir Daten zu dem Endpunkt senden und eine Vorhersage bzw Klassifkation erhalten.

First we'll need to determine how we pass data into and receive data from our endpoint. Our data is currently stored as NumPy arrays in memory of our notebook instance. To send it in an HTTP POST request, we'll serialize it as a CSV string and then decode the resulting CSV.


In [ ]:
from sagemaker.serializers import CSVSerializer
xgb_endpoint = xgb.deploy(
    endpoint_name = 'DEMO-xgboost-dm',
    initial_instance_count = 1,                    
    instance_type = 'ml.m4.xlarge',
    serializer = CSVSerializer()
)

In [ ]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_endpoint.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.drop(['y_no', 'y_yes'], axis=1).to_numpy())
predictions

Es gibt viele Möglichkeiten die Performance von XGBoost zu visualisieren und zu messen: Wir können z.B. mit einer Confusion Matrix erkennen, in welchem Verhältnis Vorhersagen und tatsächliche Werte ("ground truth") stehen.

In [ ]:
pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

Von den ~4000 potenziellen Kunden haben wir also vorhergesagt, dass 136 ein Abonnement abschließen würden, und 94 von ihnen haben es tatsächlich getan. Wir hatten auch 389 Abonnenten, die sich angemeldet haben, die wir nicht vorhergesagt haben. Das ist weniger als wünschenswert, aber das Modell kann (und sollte) abgestimmt werden, um dies zu verbessern

Für jede Stichprobe liefert unser binärer Klassifikator eine Wahrscheinlichkeit zwischen 0 und 1. Da wir uns entschieden haben, die Genauigkeit zu maximieren, legt das Modell einen Schwellenwert von 0,5 fest: Alles, was darunter liegt, wird als 0 behandelt, alles, was darüber liegt, als 1.

Um ein wenig tiefer einzutauchen: Der Schwellenwert ist in der Metrik enthalten, die XGBoost verwendet. Hier verwenden wir die Standard-Metrik "eval_metric" für die Klassifizierung, d. h. "error". Diese Metrik hat einen Standard-Schwellenwert von 0,5. Wenn Sie sich die XGBoost-Dokumentation ansehen (https://xgboost.readthedocs.io/en/latest/parameter.html), werden Sie sehen, dass es möglich ist, einen anderen Schwellenwert zu übergeben, indem Sie etwas wie folgt tun: xgb.set_hyperparameters(objective='binary:logistic', num_round=100, eval_metric='error@0.2')

## Variante 2: Vorhersagen mit Batch Transform Jobs
**Zur Erinnerung: test.csv sollte lediglich die features enthalten, jedoch nicht die Zielvariable y. Ebenso sollte die Datei keine Header row enthalten**

Manche Anwendungen benötigen keine HTTPS-basierende real-time Vorhersagen - z.B. einmal die Woche 10GB Daten zu verarbeiten und Vorhersagen zu treffen mit einem endpoint wäre nicht effizient.
Für solche Fehler gibt es die Möglichkeit SageMaker Batch Transforms zu nutzen: Wir kreiren ein Transformer Objekt und senden die Daten von S3 und erhalten ebenfalls die Vorhersagen in S3.

In [ ]:
transformer = xgb.transformer(instance_count=1, instance_type='ml.m4.xlarge')

transformer.transform('s3://{}/{}/test/test.csv'.format(bucket, prefix), content_type='text/csv')

In [ ]:
transformer.wait()
print("Die Batch-Vorhersage wurde hier gespeichert {}".format(transformer.output_path))

### Batch Vorhersagen in S3
Der Output eines Batch Transforms landet in S3:

In [ ]:
pd.read_csv(transformer.output_path+'/test.csv.out',header=None )

## Löschen des Endpunktes
Sobald wir fertig sind mit dieser Demo sollten wir den Endpoint löschen, um unnötige Kosten zu vermeiden:

In [ ]:
sagemaker.Session().delete_endpoint(xgb_endpoint.endpoint_name)